In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [ ]:
"""
    Define the tool tip offset
    Using the Clear Pen for Calibration Plate Registration
    the tool tip offset value of Medtronic Chicken Foot is from Mike github repo
    Alternatively, the tool tip offset can be calculated by PIVOTING using NDI ToolBox Utilities
"""
CLEAR_PEN_TOOLTIP_OFFSET = np.array([-1.62, 2.58, 17.80, 1]) 

In [2]:
"""
    Initialize the NDI Aurora EM tracker (Attached to Clear Pen)
"""
SETTINGS_PEN = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker_pen = NDITracker(SETTINGS_PEN)
tracker_pen.start_tracking()

In [3]:
def get_tooltip_data(tracker, tool_tip_offset):
    port_handles, timestamps, framenumbers, tracking, quality = tracker.get_frame()
    tool_tip = np.dot(tracking[0], tool_tip_offset)
    x_pos = tool_tip[0]
    y_pos = tool_tip[1]
    z_pos = tool_tip[2]
    return x_pos, y_pos, z_pos

In [4]:
"""
    Load the Markers Locations on Calibration Plate
    The locations here are in Calibration Plate's coordinate system {CP}
    The locations can be obtained by loading the Plate's .stl model into SLICER and place markers on predefined locations,
    or directly getting from .stl model in Solidworks or similar software
"""
markers_fcsv_file = 'MARKERS.fcsv'
markers_CP = pd.read_csv("C:\\Users\\f007wsq\\Desktop\\Calibration Protocol - Aurora\\Calibration Protocol" + "\\"  + markers_fcsv_file, comment='#', header=None)

In [ ]:
""""
    markers_FG saves the Marker Locations in Aurora Field Generator's coord system {FG}
    Use the Clear Pen Tool
    Point the tool tip at each marker and hit ENTER to collect the positions
"""

markers_FG = []
i = 0
while i < len(markers_CP):
    user_input = input("Press Enter to Collect Points：")
        
    if user_input.lower() == 'Exit':
        break

    if user_input == '':  # 如果按回车键
        x, y, z = get_tooltip_data(tracker=tracker_pen, tool_tip_offset=CLEAR_PEN_TOOLTIP_OFFSET)  # 获取当前点在FG中的位置
        if np.isnan(x) or np.isnan(y) or np.isnan(z):
            print("Collect again")
        else:
            markers_FG.append([x, y, z])  # 保存采集的数据       
            i = i + 1
            print(f"Collected Point: {i}")
            print(f"Position: {x, y, z}")
            

In [6]:
from scipy.spatial.transform import Rotation
def compute_rigid_transform(P_CP, P_FG):
    """
        Calculate the rigid transformation from Calibration Plate coord system {CP} to Aurora Field Generator coord system {FG}, in other word,
        Express {CP} in {FG},
        P_CP: ndarray (N,3), the markers 3D location in {CP}
        P_FG: ndarray (N,3), the markers 3D location in {FG}

        :return: R (3x3 rotation matrix), T (3x1 translation vector)
    """
    # 计算两个点集的质心
    centroid_CP = np.mean(P_CP, axis=0)
    centroid_FG = np.mean(P_FG, axis=0)

    # 去中心化（中心化点集）
    P_CP_centered = P_CP - centroid_CP
    P_FG_centered = P_FG - centroid_FG

    # 计算协方差矩阵 H
    H = P_CP_centered.T @ P_FG_centered

    # 进行 SVD 分解
    U, _, Vt = np.linalg.svd(H)
    
    # 计算旋转矩阵 R
    R = Vt.T @ U.T

    # 处理可能的反射问题（保证 R 是正交矩阵，det(R) = 1）
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # 计算平移向量 T
    T = centroid_FG - R @ centroid_CP

    return R, T

In [ ]:
P_CP = []
markers_CP = np.array(markers_CP)
for i in range(len(markers_CP)):
    P_CP.append(markers_CP[i][1:4])
P_CP = np.array(P_CP, dtype=np.float64)
print(P_CP)   

In [ ]:
P_FG = np.array(markers_FG, dtype=np.float64)
print(P_FG)

In [9]:
"""
    Calculate the Rotation and Translation
"""
R, T = compute_rigid_transform(P_CP, P_FG)

In [11]:
def create_transformation_matrix(R, T):
    """
        Combine the Rotation and Translation into 4 x 4 Transformation Matrix
    """
    transformation_matrix = np.eye(4)  # 创建一个 4x4 单位矩阵
    transformation_matrix[:3, :3] = R  # 设置旋转部分
    transformation_matrix[:3, 3] = T.flatten()  # 设置平移部分
    return transformation_matrix

In [ ]:
"""
    Here we have the transformation,
    Express {CP} in {FG} or FG to CP
    By applying the Transformation, we can have the Calibration Plate in Aurora Field Generator Coordinate System
"""
FG_to_CP_calibration = create_transformation_matrix(R, T)
print(FG_to_CP_calibration)

# Save the Transformation Matrix FG_to_CP_calibration
np.save("FG_to_CP_calibration.npy", FG_to_CP_calibration)